In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
len(text)

1115394

In [5]:
text[:123]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rathe'

In [7]:
print(text[:123])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rathe


In [13]:
set(text)

{'\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [15]:
chars = sorted(list(set(text)))

In [17]:
vocabsize = len(chars)

In [19]:
''.join(chars)

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [21]:
vocabsize

65

In [25]:
stoi = {j:i for i,j in enumerate(chars)}

In [27]:
itos = {i:j for i,j in enumerate(chars)}

In [31]:
encode = lambda s:[stoi[c] for c in s]

In [33]:
encode("Stength")

[31, 58, 43, 52, 45, 58, 46]

In [39]:
decode = lambda l:''.join([itos[c] for c in l])

In [41]:
decode([31, 58, 43, 52, 45, 58, 46])

'Stength'

In [43]:
import torch

In [45]:
torch.tensor([31, 58, 43, 52, 45, 58, 46])

tensor([31, 58, 43, 52, 45, 58, 46])

In [47]:
data = torch.tensor(encode(text) , dtype = torch.long)

In [49]:
data.shape

torch.Size([1115394])

In [51]:
data[:12]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43])

In [53]:
text[:12]

'First Citize'

In [55]:
encode('F')

[18]

In [57]:
stoi['F']

18

In [59]:
0.9*len(data)

1003854.6

In [61]:
n = int(0.9*len(data)) # 90% will be train data

In [63]:
train_data = data[:n]
val_data =data[n:]

In [65]:
# we only train with chunks of data not the entire data set
# chunk = block size

In [67]:
block_size =8
train_data[:block_size+1] 

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [69]:
x = train_data[:block_size]
y = train_data[:block_size+1] 

In [73]:
for n in range(2):
    print(n)

0
1


In [75]:
x[0]

tensor(18)

In [77]:
x[:1]

tensor([18])

In [83]:
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"input {context} and the target is {target}")

input tensor([18]) and the target is 18
input tensor([18, 47]) and the target is 47
input tensor([18, 47, 56]) and the target is 56
input tensor([18, 47, 56, 57]) and the target is 57
input tensor([18, 47, 56, 57, 58]) and the target is 58
input tensor([18, 47, 56, 57, 58,  1]) and the target is 1
input tensor([18, 47, 56, 57, 58,  1, 15]) and the target is 15
input tensor([18, 47, 56, 57, 58,  1, 15, 47]) and the target is 47


In [85]:
import torch.nn as nn

In [137]:
some = nn.Embedding(19,19) 

In [103]:
some = nn.Embedding(5,5) 

In [139]:
some

Embedding(19, 19)

In [170]:
some(x[:1]) # logits

tensor([[-0.1703, -0.5275, -1.3014, -0.5515, -0.8238, -0.2609,  0.6238,  0.3719,
         -0.7105, -0.5664,  0.6888, -0.0873,  0.1912,  1.2089,  0.3411,  0.7537,
         -0.8407, -0.2893,  0.4662]], grad_fn=<EmbeddingBackward0>)

In [101]:
x[:1]

tensor([18])

In [143]:
#nn.Embedding(19,19) :  some random embeddings of size 
# some(x[:1]) = some(18) = emb(18) = C[18] = C = torch.randn((27,2))

In [145]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

In [147]:
torch.randint(32 - 8, (8,))

tensor([ 7, 13, 20, 16, 23,  1, 23,  2])

In [154]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')

In [156]:
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[50, 39, 52, 63,  1, 47, 58, 57],
        [56, 53, 63,  1, 42, 47, 42,  1],
        [39, 51,  1, 39, 44, 56, 39, 47],
        [17, 24, 21, 38, 13, 14, 17, 32]])
targets:
torch.Size([4, 8])
tensor([[39, 52, 63,  1, 47, 58, 57, 43],
        [53, 63,  1, 42, 47, 42,  1, 57],
        [51,  1, 39, 44, 56, 39, 47, 42],
        [24, 21, 38, 13, 14, 17, 32, 20]])


In [164]:
from torch.nn import functional as F

In [168]:
class Bigram(nn.Module):
    def __init__(self, vocabsize):
        super().__init__()
        self.token_emb = nn.Embedding(vocabsize, vocabsize)
    def forward(self, ip , targets ):
        logits = self.token_emb(ip)

        loss = F.cross_entropy(logits, targets)
        # prob = logits > exp(logits)/exp(logits).sum()==probs > prob[y].log()/mean  

        return logits , loss


m = Bigram(vocabsize)
m(xb, yb)

RuntimeError: Expected target size [4, 65], got [4, 8]

In [172]:
class Bigram(nn.Module):
    def __init__(self, vocabsize):
        super().__init__()
        self.token_emb = nn.Embedding(vocabsize, vocabsize)
    def forward(self, ip , targets ):
        logits = self.token_emb(ip)
        B, T ,C = logits.shape
        logits = logits.view(B*T,C)
        targets = targets.view(B*T)

        loss = F.cross_entropy(logits, targets)
        # prob = logits > exp(logits)/exp(logits).sum()==probs > prob[y].log()/mean  

        return logits , loss


m = Bigram(vocabsize)
m(xb, yb)

(tensor([[-0.4992, -0.5556, -0.9414,  ...,  0.5055, -1.1229, -1.3802],
         [ 1.1147,  0.7647,  0.5261,  ..., -0.9591, -1.1795,  0.6583],
         [-1.2560,  1.6354, -0.1907,  ..., -1.1943, -1.0856, -0.1341],
         ...,
         [-0.8533,  0.9364,  1.1155,  ..., -1.3117,  0.9402, -1.3130],
         [-0.7666, -1.9362, -0.4154,  ..., -2.0552,  0.0795, -0.0468],
         [ 1.5458,  0.9108,  1.2844,  ...,  0.1039, -0.7675, -2.0384]],
        grad_fn=<ViewBackward0>),
 tensor(4.3659, grad_fn=<NllLossBackward0>))

In [182]:
import math
math.log(1/65)

-4.174387269895637

In [184]:
class Bigram(nn.Module):
    def __init__(self, vocabsize):
        super().__init__()
        self.token_emb = nn.Embedding(vocabsize, vocabsize)
    def forward(self, ip , targets=None ):
        logits = self.token_emb(ip)

        if targets is None:
            loss = None
        else:
            B, T ,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
    
            loss = F.cross_entropy(logits, targets)
        # prob = logits > exp(logits)/exp(logits).sum()==probs > prob[y].log()/mean  

        return logits , loss
    def generate(self, ip, max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss = self(ip)
            logits = logits[:,-1,:] # B,C
            probs = F.softmax(logits,dim=1)
            ip_next = torch.multinomial(probs,num_samples=1)
            ip = torch.cat((ip,ip_next),dim=1)
        return ip

m = Bigram(vocabsize)
m(xb, yb)

(tensor([[ 0.6172,  0.4670, -1.2181,  ...,  0.5254,  1.9695, -2.1321],
         [ 0.2342, -1.9451,  1.4819,  ...,  0.2049, -0.4716, -1.7436],
         [ 0.1686, -0.2249,  2.1984,  ..., -0.0740, -0.7982,  0.9283],
         ...,
         [-0.2485, -0.3359,  0.2612,  ..., -0.1945, -0.5056, -1.9642],
         [-0.1597, -1.5145,  1.5969,  ...,  0.9335,  0.1614, -2.3761],
         [ 1.0251, -2.6250, -1.5253,  ..., -0.3634, -1.1481,  0.6194]],
        grad_fn=<ViewBackward0>),
 tensor(5.0191, grad_fn=<NllLossBackward0>))

In [188]:
print(decode(m.generate(ip = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


ZQNhqCJ
yYSAqbs$:y's,:mrSwgt
p'xWSev-sHERhM-z&AcO'NDUTW hCk33!LfF?wuCwTvaKgC3n!QouPsv Vew?iSeitYMrw-


In [192]:
torch.zeros((1, 1), dtype=torch.long)

tensor([[0]])

In [196]:
decode([0])

'\n'

In [198]:
class Bigram(nn.Module):
    def __init__(self, vocabsize):
        super().__init__()
        self.token_emb = nn.Embedding(vocabsize, vocabsize)
    def forward(self, ip , targets=None ):
        logits = self.token_emb(ip)

        if targets is None:
            loss = None
        else:
            B, T ,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
    
            loss = F.cross_entropy(logits, targets)
        # prob = logits > exp(logits)/exp(logits).sum()==probs > prob[y].log()/mean  

        return logits , loss
    def generate(self, ip, max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss = self(ip)
            logits = logits[:,-1,:] # B,C
            print(logits)
            probs = F.softmax(logits,dim=1)
            print(probs)
            ip_next = torch.multinomial(probs,num_samples=1)
            print(ip_next)
            ip = torch.cat((ip,ip_next),dim=1)
            print(ip)
        return ip

m = Bigram(vocabsize)
m(xb, yb)

(tensor([[ 1.2242, -1.3448,  1.0039,  ..., -1.2887,  0.9148, -1.1814],
         [ 0.2700,  0.5150,  2.3193,  ..., -0.0704, -1.3662, -0.2126],
         [-0.1900,  0.5618,  0.3003,  ..., -1.1355, -1.2173,  0.1022],
         ...,
         [-0.5602, -0.7079,  0.6190,  ...,  0.1465, -1.7353, -0.2495],
         [ 0.0817,  0.4818,  0.0517,  ..., -0.7088,  2.7309, -0.6004],
         [-0.5982,  2.0037, -0.2080,  ...,  0.1363, -2.4209, -2.4576]],
        grad_fn=<ViewBackward0>),
 tensor(4.4550, grad_fn=<NllLossBackward0>))

In [200]:
print(decode(m.generate(ip = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=10)[0].tolist()))

tensor([[ 0.7926, -0.4934, -0.5571,  0.3043, -0.3415,  0.0370, -0.3436,  1.2193,
         -1.9131, -0.7418, -0.5473, -0.1534,  0.5742,  0.4423, -0.0723, -0.3840,
         -1.6930, -0.8391, -1.0826,  0.0718, -0.7146, -0.4216,  0.0734,  0.4637,
         -0.5090, -0.7130, -1.0143,  0.3516, -1.4997, -0.2790,  1.3459,  0.3984,
         -2.0686,  0.6025, -0.1272,  0.8449, -1.3304, -1.4165, -0.4636,  1.5375,
         -0.5974, -0.5451, -0.5291,  0.2046, -0.1251, -0.1885,  0.6401,  1.6612,
          0.6052, -0.3046, -0.0848, -0.1670,  1.2124, -0.1693,  0.3510,  1.9829,
         -0.0544,  1.3869, -0.4092, -0.6779,  0.3427, -1.5989,  0.8889,  0.1382,
          0.8610]], grad_fn=<SliceBackward0>)
tensor([[0.0256, 0.0071, 0.0066, 0.0157, 0.0082, 0.0120, 0.0082, 0.0392, 0.0017,
         0.0055, 0.0067, 0.0099, 0.0206, 0.0180, 0.0108, 0.0079, 0.0021, 0.0050,
         0.0039, 0.0125, 0.0057, 0.0076, 0.0125, 0.0184, 0.0070, 0.0057, 0.0042,
         0.0165, 0.0026, 0.0088, 0.0445, 0.0173, 0.0015, 0.0212

In [231]:
class Bigram(nn.Module):
    def __init__(self, vocabsize):
        super().__init__()
        self.token_emb = nn.Embedding(vocabsize, vocabsize)
    def forward(self, ip , targets=None ):
        logits = self.token_emb(ip)

        if targets is None:
            loss = None
        else:
            B, T ,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
    
            loss = F.cross_entropy(logits, targets)
        # prob = logits > exp(logits)/exp(logits).sum()==probs > prob[y].log()/mean  

        return logits , loss
    def generate(self, ip, max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss = self(ip)
            logits = logits[:,-1,:] # B,C
            probs = F.softmax(logits,dim=1)
            ip_next = torch.multinomial(probs,num_samples=1)
            ip = torch.cat((ip,ip_next),dim=1)
        return ip

m = Bigram(vocabsize)
m(xb, yb)

(tensor([[-1.8310,  0.1300, -1.1649,  ...,  0.5579, -0.4868,  0.4926],
         [-0.4426,  2.3768, -0.1844,  ...,  2.0336, -0.8147, -1.1484],
         [-0.7087, -1.5436, -0.4889,  ...,  1.4503, -0.4070, -0.8518],
         ...,
         [-1.3701, -1.1324, -1.2795,  ..., -1.3980,  0.5840, -0.1136],
         [ 0.1023,  0.2324, -0.4771,  ..., -1.7406, -1.1865, -0.6552],
         [-0.6161,  0.1592, -0.0418,  ..., -1.1270,  0.2535,  0.2040]],
        grad_fn=<ViewBackward0>),
 tensor(4.6431, grad_fn=<NllLossBackward0>))

In [239]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [241]:
batch_size = 32

In [243]:
for steps in range(10000):
    xb,yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss)

    

tensor(2.4092, grad_fn=<NllLossBackward0>)


In [245]:
print(decode(m.generate(ip = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))



RDYDWh d hepe tivedst s thes ghuanghbre thil sut oure, tl ouer EROLOUSTIw ourpery ANCI rass,
Wid?
F


In [249]:
device = 'cuda'

In [251]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y

xb, yb = get_batch('train')

In [253]:
m = Bigram(vocabsize)
m.to(device)

Bigram(
  (token_emb): Embedding(65, 65)
)

In [257]:
print(decode(m.generate(ip = torch.zeros((1, 1), dtype=torch.long, device=device), max_new_tokens=100)[0].tolist()))


?Yfz?AikRZkNwc.wj,ZTkO-FT,yCtK
q:!rLCkPBbqBUHe.XSvgO-3 SMGj?gc3aUqXbYH AtQgMNuTq&JthG.tSfFddXl!hHaLe


## Attention

In [260]:
B,T,C = 4,8,2
x=torch.randn(B,T,C)


In [262]:
x

tensor([[[-0.7547, -0.2010],
         [-0.3782,  1.0409],
         [-1.8695,  1.4091],
         [ 0.5646, -0.6764],
         [ 0.5017,  0.3483],
         [ 0.2702, -1.6603],
         [ 1.0728,  0.6532],
         [-0.3484, -0.2632]],

        [[ 0.9006, -1.8652],
         [-0.5501, -0.6626],
         [ 0.7592,  0.1089],
         [ 2.0645, -1.2541],
         [ 0.3956,  0.4751],
         [-0.0936, -0.5730],
         [-0.9944,  1.8074],
         [ 0.2255,  0.1344]],

        [[-0.2598, -0.3317],
         [ 0.2633, -0.4496],
         [-1.4718, -0.9949],
         [-1.7038, -0.7425],
         [ 1.1787, -0.0136],
         [ 0.0084, -1.1063],
         [ 0.0781,  0.3678],
         [-0.3091, -0.5354]],

        [[-0.4665, -1.3891],
         [-0.5641, -1.1680],
         [ 1.2953, -1.1608],
         [-0.6370, -1.0931],
         [ 0.5531,  0.2289],
         [ 2.4018, -0.1149],
         [-0.8258, -0.5164],
         [ 0.1412,  1.5448]]])

In [266]:
x.shape

torch.Size([4, 8, 2])

In [268]:
# one way to communicate the tokens is to average

In [272]:
x[1,2+1]

tensor([ 2.0645, -1.2541])

In [274]:
x[1,:2+1]

tensor([[ 0.9006, -1.8652],
        [-0.5501, -0.6626],
        [ 0.7592,  0.1089]])

In [276]:
for t in range(T):
    xprev = x[1,:t+1]

In [280]:
x[1]

tensor([[ 0.9006, -1.8652],
        [-0.5501, -0.6626],
        [ 0.7592,  0.1089],
        [ 2.0645, -1.2541],
        [ 0.3956,  0.4751],
        [-0.0936, -0.5730],
        [-0.9944,  1.8074],
        [ 0.2255,  0.1344]])

In [278]:
xprev

tensor([[ 0.9006, -1.8652],
        [-0.5501, -0.6626],
        [ 0.7592,  0.1089],
        [ 2.0645, -1.2541],
        [ 0.3956,  0.4751],
        [-0.0936, -0.5730],
        [-0.9944,  1.8074],
        [ 0.2255,  0.1344]])

In [286]:
xbow = torch.randn(B,T,C)

In [296]:
xbow[1]

tensor([[ 0.9006, -1.8652],
        [ 0.1752, -1.2639],
        [ 0.3699, -0.8063],
        [ 0.7935, -0.9183],
        [ 0.7139, -0.6396],
        [ 0.5793, -0.6285],
        [ 0.3545, -0.2805],
        [ 0.3384, -0.2286]])

In [290]:
for t in range(T):
    xprev = x[1,:t+1]
    xbow[1,t] = torch.mean(xprev,0)

In [294]:
xbow[1]

tensor([[ 0.9006, -1.8652],
        [ 0.1752, -1.2639],
        [ 0.3699, -0.8063],
        [ 0.7935, -0.9183],
        [ 0.7139, -0.6396],
        [ 0.5793, -0.6285],
        [ 0.3545, -0.2805],
        [ 0.3384, -0.2286]])

In [298]:
xbow = torch.randn(B,T,C) #bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b,t] = torch.mean(xprev,0)

In [300]:
x[0]

tensor([[-0.7547, -0.2010],
        [-0.3782,  1.0409],
        [-1.8695,  1.4091],
        [ 0.5646, -0.6764],
        [ 0.5017,  0.3483],
        [ 0.2702, -1.6603],
        [ 1.0728,  0.6532],
        [-0.3484, -0.2632]])

In [302]:
xbow[0]

tensor([[-0.7547, -0.2010],
        [-0.5665,  0.4199],
        [-1.0008,  0.7496],
        [-0.6095,  0.3931],
        [-0.3872,  0.3842],
        [-0.2777,  0.0434],
        [-0.0847,  0.1305],
        [-0.1177,  0.0813]])

In [304]:
# Here we are using for loops and it is computationally expensive

In [306]:
# we use @ dot product

### MAsked Attention

In [313]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = (torch.ones(3, 3))
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [315]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [317]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [323]:
wei = torch.tril(torch.ones(T, T))
wei

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [325]:
wei = wei/wei.sum(1, keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [327]:
x

tensor([[[-0.7547, -0.2010],
         [-0.3782,  1.0409],
         [-1.8695,  1.4091],
         [ 0.5646, -0.6764],
         [ 0.5017,  0.3483],
         [ 0.2702, -1.6603],
         [ 1.0728,  0.6532],
         [-0.3484, -0.2632]],

        [[ 0.9006, -1.8652],
         [-0.5501, -0.6626],
         [ 0.7592,  0.1089],
         [ 2.0645, -1.2541],
         [ 0.3956,  0.4751],
         [-0.0936, -0.5730],
         [-0.9944,  1.8074],
         [ 0.2255,  0.1344]],

        [[-0.2598, -0.3317],
         [ 0.2633, -0.4496],
         [-1.4718, -0.9949],
         [-1.7038, -0.7425],
         [ 1.1787, -0.0136],
         [ 0.0084, -1.1063],
         [ 0.0781,  0.3678],
         [-0.3091, -0.5354]],

        [[-0.4665, -1.3891],
         [-0.5641, -1.1680],
         [ 1.2953, -1.1608],
         [-0.6370, -1.0931],
         [ 0.5531,  0.2289],
         [ 2.4018, -0.1149],
         [-0.8258, -0.5164],
         [ 0.1412,  1.5448]]])

In [329]:
xbow2 = wei@x

In [331]:
xbow2

tensor([[[-7.5475e-01, -2.0099e-01],
         [-5.6649e-01,  4.1993e-01],
         [-1.0008e+00,  7.4965e-01],
         [-6.0947e-01,  3.9314e-01],
         [-3.8724e-01,  3.8416e-01],
         [-2.7767e-01,  4.3414e-02],
         [-8.4742e-02,  1.3053e-01],
         [-1.1770e-01,  8.1312e-02]],

        [[ 9.0056e-01, -1.8652e+00],
         [ 1.7521e-01, -1.2639e+00],
         [ 3.6987e-01, -8.0631e-01],
         [ 7.9353e-01, -9.1826e-01],
         [ 7.1394e-01, -6.3958e-01],
         [ 5.7935e-01, -6.2849e-01],
         [ 3.5453e-01, -2.8050e-01],
         [ 3.3840e-01, -2.2864e-01]],

        [[-2.5981e-01, -3.3167e-01],
         [ 1.7636e-03, -3.9064e-01],
         [-4.8942e-01, -5.9206e-01],
         [-7.9303e-01, -6.2967e-01],
         [-3.9868e-01, -5.0645e-01],
         [-3.3084e-01, -6.0642e-01],
         [-2.7242e-01, -4.6725e-01],
         [-2.7700e-01, -4.7577e-01]],

        [[-4.6654e-01, -1.3891e+00],
         [-5.1532e-01, -1.2786e+00],
         [ 8.8220e-02, -1.2393e+

In [335]:
xbow2[0] ,xbow[0]

(tensor([[-0.7547, -0.2010],
         [-0.5665,  0.4199],
         [-1.0008,  0.7496],
         [-0.6095,  0.3931],
         [-0.3872,  0.3842],
         [-0.2777,  0.0434],
         [-0.0847,  0.1305],
         [-0.1177,  0.0813]]),
 tensor([[-0.7547, -0.2010],
         [-0.5665,  0.4199],
         [-1.0008,  0.7496],
         [-0.6095,  0.3931],
         [-0.3872,  0.3842],
         [-0.2777,  0.0434],
         [-0.0847,  0.1305],
         [-0.1177,  0.0813]]))

### Xbow is normal averaring and Xbow is using identity and matrix multiplication averaging

In [339]:
## Another method

In [343]:
tril = torch.tril(torch.ones(T, T))

In [345]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [353]:
wei =(torch.zeros(T, T))
wei

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [355]:
wei.masked_fill(tril == 0,float('-inf'))

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [357]:
wei =  wei.masked_fill(tril == 0,float('-inf'))

In [359]:
wei = F.softmax(wei,dim=1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [361]:
#softmax = exp()/sum

In [363]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # past cannot communicate
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [369]:
# weighted aggregation 

In [367]:
a = torch.tensor([1.00001, 2.0, 3.0])
b = torch.tensor([1.00001, 2.0, 3.0])

torch.allclose(a, b)

True

In [379]:
n_embd =32
class Bigram(nn.Module):
    def __init__(self, vocabsize):
        super().__init__()
        self.token_emb_table = nn.Embedding(vocabsize, n_embd)
        self.lm_head = nn.Linear(n_embd , vocabsize)
        
    def forward(self, ip , targets=None ):
        token_emb = self.token_emb_table(ip)
        logits = self.lm_head(token_emb)

        if targets is None:
            loss = None
        else:
            B, T ,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
    
            loss = F.cross_entropy(logits, targets)
        # prob = logits > exp(logits)/exp(logits).sum()==probs > prob[y].log()/mean  

        return logits , loss
    def generate(self, ip, max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss = self(ip)
            logits = logits[:,-1,:] # B,C
            probs = F.softmax(logits,dim=1)
            ip_next = torch.multinomial(probs,num_samples=1)
            ip = torch.cat((ip,ip_next),dim=1)
        return ip

m = Bigram(vocabsize)
m.to(device)

Bigram(
  (token_emb_table): Embedding(65, 32)
  (lm_head): Linear(in_features=32, out_features=65, bias=True)
)

In [381]:
n_embd =32
class Bigram(nn.Module):
    def __init__(self, vocabsize):
        super().__init__()
        self.token_emb_table = nn.Embedding(vocabsize, n_embd)
        self.position_emb_table = nn.Embedding(block_size, n_embd)
        self.lm_head = nn.Linear(n_embd , vocabsize)
        
    def forward(self, ip , targets=None ):
        B, t = ip.shape
        
        token_emb = self.token_emb_table(ip)
        pos_emb = self.position_emb_table(torch.arange(T, device = device)) # (T,C)
        x = token_emb + pos_emb
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T ,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
    
            loss = F.cross_entropy(logits, targets)
        # prob = logits > exp(logits)/exp(logits).sum()==probs > prob[y].log()/mean  

        return logits , loss
    def generate(self, ip, max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss = self(ip)
            logits = logits[:,-1,:] # B,C
            probs = F.softmax(logits,dim=1)
            ip_next = torch.multinomial(probs,num_samples=1)
            ip = torch.cat((ip,ip_next),dim=1)
        return ip

m = Bigram(vocabsize)
m.to(device)

Bigram(
  (token_emb_table): Embedding(65, 32)
  (position_emb_table): Embedding(8, 32)
  (lm_head): Linear(in_features=32, out_features=65, bias=True)
)

### Self Attention

In [384]:
torch.manual_seed(1337)
B,T,C = 4,8,32
x=torch.randn(B,T,C)


In [388]:
x

tensor([[[ 0.1808, -0.0700, -0.3596,  ..., -0.8016,  1.5236,  2.5086],
         [-0.6631, -0.2513,  1.0101,  ...,  1.5333,  1.6097, -0.4032],
         [-0.8345,  0.5978, -0.0514,  ..., -0.4370, -1.0012, -0.4094],
         ...,
         [-0.8961,  0.0662, -0.0563,  ...,  2.1382,  0.5114,  1.2191],
         [ 0.1910, -0.3425,  1.7955,  ...,  0.3699, -0.5556, -0.3983],
         [-0.5819, -0.2208,  0.0135,  ..., -1.9079, -0.5276,  1.0807]],

        [[ 0.4562, -1.0917, -0.8207,  ...,  0.0512, -0.6576, -2.5729],
         [ 0.0210,  1.0060, -1.2492,  ...,  0.7859, -1.1501,  1.3132],
         [ 2.2007, -0.2195,  0.5427,  ..., -0.6445,  1.0834, -0.7995],
         ...,
         [ 0.3091,  1.1661, -2.1821,  ...,  0.6151,  0.6763,  0.6228],
         [ 0.0943, -0.3156,  0.7850,  ..., -1.5735,  1.3876,  0.7251],
         [ 0.6455, -0.3313, -1.0390,  ...,  0.0895, -0.3748, -0.4781]],

        [[-0.6067,  1.8328,  0.2931,  ...,  1.0041,  0.8656,  0.1688],
         [-0.2352, -0.2586,  0.0131,  ...,  0

In [386]:
head_size = 16
key = nn.Linear(C, head_size, bias =False) # C =32 >> head_size=16
query = nn.Linear(C, head_size, bias=False)
k = key(x) # (B,T,16)
q= query(x)

In [394]:
x.shape

torch.Size([4, 8, 32])

In [392]:
k.shape

torch.Size([4, 8, 16])

In [396]:
k.transpose(-2,-1).shape

torch.Size([4, 16, 8])

In [398]:
q.shape

torch.Size([4, 8, 16])

In [402]:
wei = q @ k.transpose(-2,-1)  #(B,T 16) @ (B, 16, T) --> (B,T,T)

In [406]:
tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # past cannot communicate
wei = F.softmax(wei, dim=-1) #normalize
out = wei @ x

In [408]:
out.shape

torch.Size([4, 8, 32])

In [412]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3351, 0.6649, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2877, 0.2753, 0.4370, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3407, 0.2150, 0.2306, 0.2138, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1604, 0.1730, 0.1632, 0.1601, 0.3434, 0.0000, 0.0000, 0.0000],
        [0.1387, 0.1784, 0.1393, 0.1375, 0.2694, 0.1366, 0.0000, 0.0000],
        [0.1455, 0.1845, 0.1284, 0.1281, 0.1364, 0.1502, 0.1270, 0.0000],
        [0.1122, 0.1195, 0.1161, 0.1382, 0.1164, 0.1179, 0.1400, 0.1396]],
       grad_fn=<SelectBackward0>)

In [414]:
# for the 8th token 0.1400 this one seems interesting

In [416]:
head_size = 16
key = nn.Linear(C, head_size, bias =False) # C =32 >> head_size=16
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B,T,16)
q= query(x)
v= value(x)


tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # past cannot communicate
wei = F.softmax(wei, dim=-1) #normalize
out = wei @ v

In [418]:
out.shape

torch.Size([4, 8, 16])

In [428]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # only encoder u delete this
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [422]:
x

tensor([[[ 0.1808, -0.0700, -0.3596,  ..., -0.8016,  1.5236,  2.5086],
         [-0.6631, -0.2513,  1.0101,  ...,  1.5333,  1.6097, -0.4032],
         [-0.8345,  0.5978, -0.0514,  ..., -0.4370, -1.0012, -0.4094],
         ...,
         [-0.8961,  0.0662, -0.0563,  ...,  2.1382,  0.5114,  1.2191],
         [ 0.1910, -0.3425,  1.7955,  ...,  0.3699, -0.5556, -0.3983],
         [-0.5819, -0.2208,  0.0135,  ..., -1.9079, -0.5276,  1.0807]],

        [[ 0.4562, -1.0917, -0.8207,  ...,  0.0512, -0.6576, -2.5729],
         [ 0.0210,  1.0060, -1.2492,  ...,  0.7859, -1.1501,  1.3132],
         [ 2.2007, -0.2195,  0.5427,  ..., -0.6445,  1.0834, -0.7995],
         ...,
         [ 0.3091,  1.1661, -2.1821,  ...,  0.6151,  0.6763,  0.6228],
         [ 0.0943, -0.3156,  0.7850,  ..., -1.5735,  1.3876,  0.7251],
         [ 0.6455, -0.3313, -1.0390,  ...,  0.0895, -0.3748, -0.4781]],

        [[-0.6067,  1.8328,  0.2931,  ...,  1.0041,  0.8656,  0.1688],
         [-0.2352, -0.2586,  0.0131,  ...,  0

In [424]:
x.shape

torch.Size([4, 8, 32])

### so here all these 4 are independent and each with communicate to them selves 

In [ ]:
#scaled attendion = if u 

### divide by sqrt(dmodel) -just to normalize or scale to control the varience at inintialization

In [430]:
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) 

In [432]:
k.var()

tensor(0.3164, grad_fn=<VarBackward0>)

In [434]:
wei.var()

tensor(1.9224, grad_fn=<VarBackward0>)

In [436]:
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) * head_size**-0.5

In [438]:
wei.var()

tensor(0.1201, grad_fn=<VarBackward0>)

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # only encoder u delete this
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

#### Self Attention Block

In [479]:
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(C, head_size, bias =False) # C =32 >> head_size=16
        self.query = nn.Linear(C, head_size, bias=False)
        self.value = nn.Linear(C, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self,x):
        B,T,C = x.shape
        k = self.key(x)   # (B, T, 16)
        q = self.query(x)
        wei =  q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        v = self.value(x)
        out = wei @ v
        return out
        
        
    

In [481]:
n_embd =32
class Bigram(nn.Module):
    def __init__(self, vocabsize):
        super().__init__()
        self.token_emb_table = nn.Embedding(vocabsize, n_embd)
        self.position_emb_table = nn.Embedding(block_size, n_embd)
        self.selfA_head = Head(n_embd)
        self.lm_head = nn.Linear(n_embd , vocabsize)
        
        
    def forward(self, ip , targets=None ):
        B, t = ip.shape
        
        token_emb = self.token_emb_table(ip)
        pos_emb = self.position_emb_table(torch.arange(T, device = device)) # (T,C)
        x = token_emb + pos_emb
        x = selfA_head(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T ,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
    
            loss = F.cross_entropy(logits, targets)
        # prob = logits > exp(logits)/exp(logits).sum()==probs > prob[y].log()/mean  

        return logits , loss
    def generate(self, ip, max_new_tokens):
        for _ in range(max_new_tokens):
            ip_cond = ip[:,-block_size:] # crop input to block size  ## as positional embeddings
            logits,loss = self(ip_cond)
            logits = logits[:,-1,:] # B,C
            probs = F.softmax(logits,dim=1)
            ip_next = torch.multinomial(probs,num_samples=1)
            ip = torch.cat((ip,ip_next),dim=1)
        return ip

m = Bigram(vocabsize)
m.to(device)

Bigram(
  (token_emb_table): Embedding(65, 32)
  (position_emb_table): Embedding(8, 32)
  (selfA_head): Head(
    (key): Linear(in_features=32, out_features=32, bias=False)
    (query): Linear(in_features=32, out_features=32, bias=False)
    (value): Linear(in_features=32, out_features=32, bias=False)
  )
  (lm_head): Linear(in_features=32, out_features=65, bias=True)
)

## Multi Attention

In [484]:
class MultiAttention(nn.Module):
    def __init__(self,num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim=-1)

In [486]:
n_embd =32
class Bigram(nn.Module):
    def __init__(self, vocabsize):
        super().__init__()
        self.token_emb_table = nn.Embedding(vocabsize, n_embd)
        self.position_emb_table = nn.Embedding(block_size, n_embd)
        self.selfA_head = MultiAttention(4, n_embd//4)
        self.lm_head = nn.Linear(n_embd , vocabsize)
        
        
    def forward(self, ip , targets=None ):
        B, T = ip.shape
        
        token_emb = self.token_emb_table(ip)
        pos_emb = self.position_emb_table(torch.arange(T, device = device)) # (T,C)
        x = token_emb + pos_emb
        x = self.selfA_head(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T ,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
    
            loss = F.cross_entropy(logits, targets)
        # prob = logits > exp(logits)/exp(logits).sum()==probs > prob[y].log()/mean  

        return logits , loss
    def generate(self, ip, max_new_tokens):
        for _ in range(max_new_tokens):
            ip_cond = ip[:,-block_size:] # crop input to block size  ## as positional embeddings
            logits,loss = self(ip_cond)
            logits = logits[:,-1,:] # B,C
            probs = F.softmax(logits,dim=1)
            ip_next = torch.multinomial(probs,num_samples=1)
            ip = torch.cat((ip,ip_next),dim=1)
        return ip

m = Bigram(vocabsize)
m.to(device)

Bigram(
  (token_emb_table): Embedding(65, 32)
  (position_emb_table): Embedding(8, 32)
  (selfA_head): MultiAttention(
    (heads): ModuleList(
      (0-3): 4 x Head(
        (key): Linear(in_features=32, out_features=8, bias=False)
        (query): Linear(in_features=32, out_features=8, bias=False)
        (value): Linear(in_features=32, out_features=8, bias=False)
      )
    )
  )
  (lm_head): Linear(in_features=32, out_features=65, bias=True)
)

In [488]:
for steps in range(10000):
    xb,yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss)

tensor(4.1822, device='cuda:0', grad_fn=<NllLossBackward0>)


### Feed Forward layer

In [494]:
#once we multiply the and get attention layer we send to FFL so that they will understand 

In [498]:
#feed forward is individual

In [492]:
class FeedForward(nn.Module):
    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd),
            nn.ReLU(),
        )
    def forward(self, x):
        return self.net(x)

In [500]:
n_embd =32
class Bigram(nn.Module):
    def __init__(self, vocabsize):
        super().__init__()
        self.token_emb_table = nn.Embedding(vocabsize, n_embd)
        self.position_emb_table = nn.Embedding(block_size, n_embd)
        self.selfA_head = MultiAttention(4, n_embd//4)
        self.lm_head = nn.Linear(n_embd , vocabsize)
        self.ffwd = FeedForward(n_embd)
        
    def forward(self, ip , targets=None ):
        B, T = ip.shape
        
        token_emb = self.token_emb_table(ip)
        pos_emb = self.position_emb_table(torch.arange(T, device = device)) # (T,C)
        x = token_emb + pos_emb
        x = self.selfA_head(x)
        x = self.ffwd(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T ,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
    
            loss = F.cross_entropy(logits, targets)
        # prob = logits > exp(logits)/exp(logits).sum()==probs > prob[y].log()/mean  

        return logits , loss
    def generate(self, ip, max_new_tokens):
        for _ in range(max_new_tokens):
            ip_cond = ip[:,-block_size:] # crop input to block size  ## as positional embeddings
            logits,loss = self(ip_cond)
            logits = logits[:,-1,:] # B,C
            probs = F.softmax(logits,dim=1)
            ip_next = torch.multinomial(probs,num_samples=1)
            ip = torch.cat((ip,ip_next),dim=1)
        return ip

m = Bigram(vocabsize)
m.to(device)

Bigram(
  (token_emb_table): Embedding(65, 32)
  (position_emb_table): Embedding(8, 32)
  (selfA_head): MultiAttention(
    (heads): ModuleList(
      (0-3): 4 x Head(
        (key): Linear(in_features=32, out_features=8, bias=False)
        (query): Linear(in_features=32, out_features=8, bias=False)
        (value): Linear(in_features=32, out_features=8, bias=False)
      )
    )
  )
  (lm_head): Linear(in_features=32, out_features=65, bias=True)
  (ffwd): FeedForward(
    (net): Sequential(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): ReLU()
    )
  )
)

In [506]:
FINE TUNE > RL

SyntaxError: invalid syntax (1919879560.py, line 1)